In [1]:
import os
from pathlib import Path
from os import path

import geopandas as gpd
import numpy as np
import pandas as pd
import scipy.stats as stats
import statsmodels.api as sm

from statsmodels.stats.outliers_influence import variance_inflation_factor

# 1. Input data

In [2]:
cwd = os.getcwd()
root_dir = Path(cwd).parent

In [3]:
## Input data

# Individual data
data_IND_tot = pd.read_csv(path.join(root_dir,
                                    "data/raw/individual_data.csv")).drop(['Unnamed: 0'],axis=1)

# Household data
data_HH_tot = pd.read_csv(path.join(root_dir,
                                    "data/raw/household_data.csv")).drop(['Unnamed: 0'],axis=1)

# 2. Preprocess data

## 2.1. Join dwelling attributes (household-level) to individual-level dataframe

In [4]:
## Join tables

# Dummy dataframes
dst_data = data_IND_tot.copy()
src_data = data_HH_tot.copy().drop(['City','Site','uuid'],axis=1)
src_data = src_data.rename(columns={"KEY": "PARENT_KEY"})

# Join
dst_data = dst_data.merge(src_data, how='left', on='PARENT_KEY')

# Check results
dst_data

,KEY,PARENT_KEY,City,Site,Relation_to_HH,Age,Sex,School_past,Diarrhoea,Sex_HH,...,HH_Recor_2_std,b_area,NN_100,n_blg_cc,n_blg_ccM,n_CAR,n_CARM,n_mn_Bdev,n_mn_BdevM,WCdirtyS
0,uuid:464423ec-b7a0-4924-b3e4-ce85bb6a5286/Rep_...,uuid:464423ec-b7a0-4924-b3e4-ce85bb6a5286,Abidjan,Azito,Head,47.0,M,Secondary_2,N,M,...,0.545887,160.283340,152,0.476315,0.521259,0.676246,0.517386,2.472029,3.534918,NaN
1,uuid:464423ec-b7a0-4924-b3e4-ce85bb6a5286/Rep_...,uuid:464423ec-b7a0-4924-b3e4-ce85bb6a5286,Abidjan,Azito,S_D,20.0,M,Secondary_2,N,M,...,0.545887,160.283340,152,0.476315,0.521259,0.676246,0.517386,2.472029,3.534918,NaN
2,uuid:b08a7cd9-67cf-45a0-9552-26956400cf7a/Rep_...,uuid:b08a7cd9-67cf-45a0-9552-26956400cf7a,Abidjan,Azito,Head,58.0,M,High_Ed,N,M,...,0.174541,171.418746,104,0.539697,0.513751,0.415604,0.556119,1.573132,2.410601,NaN
3,uuid:b08a7cd9-67cf-45a0-9552-26956400cf7a/Rep_...,uuid:b08a7cd9-67cf-45a0-9552-26956400cf7a,Abidjan,Azito,W_H,45.0,F,Secondary_2,Y,M,...,0.174541,171.418746,104,0.539697,0.513751,0.415604,0.556119,1.573132,2.410601,NaN
4,uuid:b08a7cd9-67cf-45a0-9552-26956400cf7a/Rep_...,uuid:b08a7cd9-67cf-45a0-9552-26956400cf7a,Abidjan,Azito,GndC,3.0,F,NaN,N,M,...,0.174541,171.418746,104,0.539697,0.513751,0.415604,0.556119,1.573132,2.410601,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6279,uuid:86c835c1-c642-432c-9c69-177958aa9d29/Rep_...,uuid:86c835c1-c642-432c-9c69-177958aa9d29,Nairobi,Vietnam,W_H,22.0,F,Primary,N,M,...,0.725594,101.889328,243,0.301713,0.377274,0.755996,0.732830,0.001790,2.246444,0.0
6280,uuid:007e2f7b-0d59-4de9-aab1-3509c6c4566f/Rep_...,uuid:007e2f7b-0d59-4de9-aab1-3509c6c4566f,Nairobi,Vietnam,Head,36.0,M,Secondary,N,M,...,0.503001,82.662454,252,0.578906,0.379600,0.747246,0.707976,1.842102,3.087594,0.0
6281,uuid:007e2f7b-0d59-4de9-aab1-3509c6c4566f/Rep_...,uuid:007e2f7b-0d59-4de9-aab1-3509c6c4566f,Nairobi,Vietnam,W_H,28.0,F,NaN,N,M,...,0.503001,82.662454,252,0.578906,0.379600,0.747246,0.707976,1.842102,3.087594,0.0
6282,uuid:007e2f7b-0d59-4de9-aab1-3509c6c4566f/Rep_...,uuid:007e2f7b-0d59-4de9-aab1-3509c6c4566f,Nairobi,Vietnam,S_D,9.0,M,NaN,N,M,...,0.503001,82.662454,252,0.578906,0.379600,0.747246,0.707976,1.842102,3.087594,0.0


## 2.2. Set exposure variables and subsets (age groups)

In [5]:
## Set exposure variables necessary for analysis: WASH facilities' characteristics + potential confounders

# Dummy dataframe
df = dst_data.copy()

## Outcome of interest

# Recode cases of diarrhoea
case_mask = (df["Diarrhoea"]=="Y") # participant had diarrhoea in the 2 weeks preceeding the survey
df["Case"] = np.nan
df["Case"][~df["Diarrhoea"].isna()] = 0
df["Case"][case_mask] = 1


## Exposure of interest

# Perceived safety to go to WC
wc_unsf_mask = ((df["ToiletFacilitySAFE"]=="During_Day")|
                (df["ToiletFacilitySAFE"]=="Unsafe")) # if safe only during the day, or never, toilet is considered unsafe
df["WCunsafe"] = np.nan
df["WCunsafe"][~df["ToiletFacilitySAFE"].isna()] = 0
df["WCunsafe"][wc_unsf_mask] = 1


## Sanitation and hygiene facilities

# Access to "improved" sanitation (according to WHO-UNICEF's JMP) VS any other facitilty type ("unimproved" and O.D.)
isan_mask_abi = ((((df.City=='Abidjan')&
                   (df.ToiletFacilityTYPE=='DryOrCompost')&
                   (df.ToiletFacilityTYPE_Dry.isin(['Dry_ImprSlab', # improved pit latrine
                                                    'Dry_VIP'])))| # VIP: ventilated, improved pit latrine
                  ((df.City=='Abidjan')&
                   (df.ToiletFacilityTYPE=='Flush')&
                   (df.ToiletFacilityTYPE_Flush.isin(['Flush_piped',
                                                      'Flush_septic_tank',
                                                      'Flush_coveredPit']))))&(df.ToiletROOF=='Y')) # many toilets in Abidjan did not have a roof
isan_mask_nai = ((((df.City=='Nairobi')&
                   (df.ToiletFacilityTYPE=='DryOrCompost')&
                   (df.ToiletFacilityTYPE_Dry.isin(['Dry_ImprSlab', # improved pit latrine
                                                    'Dry_VIP', # VIP: ventilated, improved pit latrine
                                                    'Dry_FreshLife'])))| # Sanergy's FreshLife toilet unit
                  ((df.City=='Nairobi')&
                   (df.ToiletFacilityTYPE=='Flush')&
                   (df.ToiletFacilityTYPE_Flush.isin(['Flush_piped',
                                                      'Flush_septic_tank',
                                                      'Flush_coveredPit'])))))
df["ImprvSan"] = np.nan
df["ImprvSan"][(~df.ToiletFacility.isna())] = 0
df["ImprvSan"][isan_mask_abi] = 1
df["ImprvSan"][isan_mask_nai] = 1

# Shared toilet
wc_share_mask = (df["ToiletFacilitySHARE"]=='Y') # toilet shared by more than one household
df["WCshared"] = np.nan
df["WCshared"][(~df["ToiletFacilitySHARE"].isna())] = 0
df["WCshared"][wc_share_mask] = 1

# Access to "basic" hygiene (according to WHO-UNICEF's JMP)
bhyg_mask_abi = ((df.City=='Abidjan')&
                 (df["ObsHandWashWATER"]=='Water_OK')& # water available at the moment of the survey
                 (df["ObsHandWashSOAP"]=='SoapOrDeterg')& # soap (or equivalent) available at the moment of the survey
                 (df["ObsHandWashPLACE"].isin(['Obs_Fixed','Obs_Mobile'])) # presence of amenity to wash hands
                )
bhyg_mask_nai = ((df.City=='Nairobi')&
                 (df["ObsHandWashWATER"]=='Water_OK')& # water available at the moment of the survey
                 (df["ObsHandWashSOAP"]=='SoapOrDeterg')& # soap (or equivalent) available at the moment of the survey
                 (df["ObsHandWashPLACE"].isin(['Obs_Fixed'])) # presence of FIXED amenity to wash hands ('mobile' amenities were not reliable in Nairobi)
                )
df["BasicHyg"] = np.nan
df["BasicHyg"][(~df["ObsHandWashWATER"].isna())&
               (~df["ObsHandWashSOAP"].isna())&
               (~df["ObsHandWashPLACE"].isna())] = 0
df["BasicHyg"][bhyg_mask_abi] = 1
df["BasicHyg"][bhyg_mask_nai] = 1


## External contamination pathways

# Recode frequency of consumption of street food
st_food_mask = (df["StreetFood"].isin(['2_4','5_Or_More'])) # twice or more per week
df["StFood"] = np.nan
df["StFood"][(~df["StreetFood"].isna())] = 0
df["StFood"][st_food_mask] = 1


## Potential confounders

# Education level of heads of households
src = data_IND_tot[data_IND_tot.Relation_to_HH=='Head'][['PARENT_KEY',
                                                         'School_past']]# subset ed. level of HH
# heads of households with at least secondary education
src['SecEduHH'] = np.nan
src['SecEduHH'][src['School_past'].isin(['No_Edu',
                                         'Early_CdE',
                                         'Primary',
                                         'Coranic'])] = 0 # up to primary education
src['SecEduHH'][src['School_past'].isin(['Secondary',
                                         'Secondary_1',
                                         'Secondary_2',
                                         'High_Ed'])] = 1 # secondary or higher education
src = src[~src.SecEduHH.isna()]
# In case a same household has 2 heads, merge lines
src = src[['SecEduHH','PARENT_KEY']].groupby(by="PARENT_KEY").max().reset_index()
# Attribute education level of heads of households
df = df.merge(src,on="PARENT_KEY",how='left')

In [6]:
## Subsets

# General population
df_abi = df[df['City']=='Abidjan'].copy()
print("N for general pop. in Abidjan:",df_abi.shape[0])
df_nai = df[df['City']=='Nairobi'].copy()
print("N for general pop. in Nairobi:",df_nai.shape[0])

# Children under five
mask = (df["Age"]<5)
df_U5 = df[mask]
df_U5_abi = df_U5[df_U5['City']=='Abidjan'].copy()
print("N for general pop. in Abidjan:",df_U5_abi.shape[0])
df_U5_nai = df_U5[df_U5['City']=='Nairobi'].copy()
print("N for general pop. in Nairobi:",df_U5_nai.shape[0])

N for general pop. in Abidjan: 2498
N for general pop. in Nairobi: 3786
N for general pop. in Abidjan: 283
N for general pop. in Nairobi: 491


# 3. Odds ratios

## 3.1. Bivariate odds ratio analysis to identify candidate explanatory variables for diarrhoea

### 3.1.1. Calculate ORs

In [7]:
## Set list of exposure variables

# List
exposure_lst = ['WCunsafe', # lack of safety to go to most used toilet
                'WCdirtyS', # most used toilet is considered dirty based on global views (at least 1/2 of users)
                'WCshared', # shared sanitation
                'StFood', # frequent consumption of street foods (twice or more per week)
                'SecEduHH' # whether head of household has secondary education
               ]

# Subsets list 1: general population
subsets_gen = [df_abi,df_nai]
subsets_gen_str = ['general population, Abidjan','general population, Nairobi']

# Subsets list 2: households with children under five years old
subsets_cU5 = [df_U5_abi,df_U5_nai]
subsets_cU5_str = ['children under five, Abidjan','children under five, Nairobi']

In [8]:
## Risk of diarrhoea, general population

# Calculate odds ratios
outcome_var = "Case"
outcome_pos = 1
outcome_neg = 0
df_oddsr_gen = pd.DataFrame()
for idx, subset in enumerate(subsets_gen):
    print("------------------ ",subsets_gen_str[idx]," ------------------")
    for exposure in exposure_lst:
        # define groups
        print("Variable:",exposure)
        exposure_grp = subset[subset[exposure]==1]
        no_exposure_grp = subset[subset[exposure]==0]
        exposure_grp = exposure_grp[~exposure_grp[outcome_var].isna()]
        no_exposure_grp = no_exposure_grp[~no_exposure_grp[outcome_var].isna()]
        # set table for Fisher tests
        table = np.array([[exposure_grp[exposure_grp[outcome_var]==outcome_pos].shape[0], exposure_grp[exposure_grp[outcome_var]==outcome_neg].shape[0]],
                          [no_exposure_grp[no_exposure_grp[outcome_var]==outcome_pos].shape[0], no_exposure_grp[no_exposure_grp[outcome_var]==outcome_neg].shape[0]]])
        # calculate proportion of households with at least 1 case
        exposure_grp_prop = exposure_grp[exposure_grp[outcome_var]==outcome_pos].shape[0]/exposure_grp.shape[0]
        no_exposure_grp_prop = no_exposure_grp[no_exposure_grp[outcome_var]==outcome_pos].shape[0]/no_exposure_grp.shape[0]
        # calculate 95% CI - exposure group
        P_exp = exposure_grp_prop
        N_exp = exposure_grp.shape[0]
        CI_exp = 1.96*(np.sqrt((P_exp*(1-P_exp))/N_exp))
        # calculate 95% CI - no exposure group
        P_ne = no_exposure_grp_prop
        N_ne = no_exposure_grp.shape[0]
        CI_ne = 1.96*(np.sqrt((P_ne*(1-P_ne))/N_ne))
        # run Fisher tests for OR = 1
        oddsratio_eq1, pvalue_eq1 = stats.fisher_exact(table)
        # run Fisher tests for OR > 1
        oddsratio_greater1, pvalue_greater1 = stats.fisher_exact(table,alternative="greater")
        # run Fisher tests for OR < 1
        oddsratio_less1, pvalue_less1 = stats.fisher_exact(table,alternative="less")
        # add results to dataframe
        df_oddsr_gen = df_oddsr_gen.append([[exposure,
                                             subsets_gen_str[idx],
                                             exposure_grp_prop*100,
                                             '±'+str(round(CI_exp*100,2)),
                                             no_exposure_grp_prop*100,
                                             '±'+str(round(CI_ne*100,2)),
                                             oddsratio_eq1,
                                             pvalue_eq1,
                                             pvalue_greater1,
                                             pvalue_less1,
                                             table
                                            ]])

# Reset columns' names & index
df_oddsr_gen.columns = ['exposure variable','stratum',
                        '% outcomes, exposed','95% CI, exposed',
                        '% outcomes, non-exposed','95% CI, non-exposed',
                        'OR','p-value for OR=1',
                        'p_OR_hi_1','p_OR_lo_1',
                        'table'
                       ]
df_oddsr_gen = df_oddsr_gen.reset_index()

------------------  general population, Abidjan  ------------------
Variable: WCunsafe
Variable: WCdirtyS
Variable: WCshared
Variable: StFood
Variable: SecEduHH
------------------  general population, Nairobi  ------------------
Variable: WCunsafe
Variable: WCdirtyS
Variable: WCshared
Variable: StFood
Variable: SecEduHH


In [9]:
## Risk of diarrhoea, under fives subsets

# Calculate odds ratios
outcome_var = "Case"
outcome_pos = 1
outcome_neg = 0
df_oddsr_cU5 = pd.DataFrame()
for idx, subset in enumerate(subsets_cU5):
    print("------------------ ",subsets_cU5_str[idx]," ------------------")
    for exposure in exposure_lst:
        # define groups
        print("Variable:",exposure)
        exposure_grp = subset[subset[exposure]==1]
        no_exposure_grp = subset[subset[exposure]==0]
        exposure_grp = exposure_grp[~exposure_grp[outcome_var].isna()]
        no_exposure_grp = no_exposure_grp[~no_exposure_grp[outcome_var].isna()]
        # set table for Fisher tests
        table = np.array([[exposure_grp[exposure_grp[outcome_var]==outcome_pos].shape[0], exposure_grp[exposure_grp[outcome_var]==outcome_neg].shape[0]],
                          [no_exposure_grp[no_exposure_grp[outcome_var]==outcome_pos].shape[0], no_exposure_grp[no_exposure_grp[outcome_var]==outcome_neg].shape[0]]])
        # calculate proportion of households with at least 1 case
        exposure_grp_prop = exposure_grp[exposure_grp[outcome_var]==outcome_pos].shape[0]/exposure_grp.shape[0]
        no_exposure_grp_prop = no_exposure_grp[no_exposure_grp[outcome_var]==outcome_pos].shape[0]/no_exposure_grp.shape[0]
        # calculate 95% CI - exposure group
        P_exp = exposure_grp_prop
        N_exp = exposure_grp.shape[0]
        CI_exp = 1.96*(np.sqrt((P_exp*(1-P_exp))/N_exp))
        # calculate 95% CI - no exposure group
        P_ne = no_exposure_grp_prop
        N_ne = no_exposure_grp.shape[0]
        CI_ne = 1.96*(np.sqrt((P_ne*(1-P_ne))/N_ne))
        # run Fisher tests for OR = 1
        oddsratio_eq1, pvalue_eq1 = stats.fisher_exact(table)
        # run Fisher tests for OR > 1
        oddsratio_greater1, pvalue_greater1 = stats.fisher_exact(table,alternative="greater")
        # run Fisher tests for OR < 1
        oddsratio_less1, pvalue_less1 = stats.fisher_exact(table,alternative="less")
        # add results to dataframe
        df_oddsr_cU5 = df_oddsr_cU5.append([[exposure,
                                             subsets_cU5_str[idx],
                                             exposure_grp_prop*100,
                                             '±'+str(round(CI_exp*100,2)),
                                             no_exposure_grp_prop*100,
                                             '±'+str(round(CI_ne*100,2)),
                                             oddsratio_eq1,
                                             pvalue_eq1,
                                             pvalue_greater1,
                                             pvalue_less1,
                                             table
                                            ]])

# Reset columns' names
df_oddsr_cU5.columns = ['exposure variable','stratum',
                        '% outcomes, exposed','95% CI, exposed',
                        '% outcomes, non-exposed','95% CI, non-exposed',
                        'OR','p-value for OR=1',
                        'p_OR_hi_1','p_OR_lo_1',
                        'table'
                       ]
df_oddsr_cU5 = df_oddsr_cU5.reset_index()

------------------  children under five, Abidjan  ------------------
Variable: WCunsafe
Variable: WCdirtyS
Variable: WCshared
Variable: StFood
Variable: SecEduHH
------------------  children under five, Nairobi  ------------------
Variable: WCunsafe
Variable: WCdirtyS
Variable: WCshared
Variable: StFood
Variable: SecEduHH


In [10]:
## List of significant covariates

# Significance threshold (maximum p-value)
st = 0.05

# General population
list_gen = df_oddsr_gen['exposure variable'][df_oddsr_gen['p-value for OR=1']<st].to_list()

# Under fives subset
list_cu5 = df_oddsr_cU5['exposure variable'][df_oddsr_cU5['p-value for OR=1']<st].to_list()

# Merge lists
merged_lst = list(set(list_gen+list_cu5))
intersect_lst = list(set(list_gen)&set(list_cu5))

# Dataframe containing only selected covariates, by age group
df_raw_or = df_oddsr_gen[df_oddsr_gen['exposure variable'].isin(merged_lst)].append(df_oddsr_cU5[df_oddsr_cU5['exposure variable'].isin(merged_lst)]).drop(['p_OR_hi_1', 'p_OR_lo_1'],axis=1)

# Check results
print("Selected covariates ( significant either in gen. pop. or under-fives, n=",len(merged_lst),"):",merged_lst)
print("Covariates significant BOTH in gen. pop. or under-fives ( n=",len(intersect_lst),"):",intersect_lst)
df_raw_or

Selected covariates ( significant either in gen. pop. or under-fives, n= 4 ): ['WCdirtyS', 'WCunsafe', 'StFood', 'SecEduHH']
Covariates significant BOTH in gen. pop. or under-fives ( n= 3 ): ['StFood', 'WCdirtyS', 'SecEduHH']


,index,exposure variable,stratum,"% outcomes, exposed","95% CI, exposed","% outcomes, non-exposed","95% CI, non-exposed",OR,p-value for OR=1,table
0,0,WCunsafe,"general population, Abidjan",26.132404,±5.08,12.642369,±2.2,2.444544,2.369003e-07,"[[75, 212], [111, 767]]"
1,0,WCdirtyS,"general population, Abidjan",24.309392,±4.42,12.640801,±2.3,2.219556,1.286825e-06,"[[88, 274], [101, 698]]"
3,0,StFood,"general population, Abidjan",14.974937,±1.75,14.099217,±2.46,1.073051,6.194800e-01,"[[239, 1357], [108, 658]]"
4,0,SecEduHH,"general population, Abidjan",16.568627,±2.28,13.405797,±2.01,1.282783,4.428440e-02,"[[169, 851], [148, 956]]"
5,0,WCunsafe,"general population, Nairobi",16.365688,±2.44,10.840210,±1.18,1.609464,2.464488e-05,"[[145, 741], [289, 2377]]"
6,0,WCdirtyS,"general population, Nairobi",17.415730,±3.22,11.310116,±1.13,1.653680,1.377044e-04,"[[93, 441], [341, 2674]]"
8,0,StFood,"general population, Nairobi",13.629097,±1.34,8.882784,±1.69,1.618643,4.973263e-05,"[[341, 2161], [97, 995]]"
9,0,SecEduHH,"general population, Nairobi",10.907945,±1.53,12.769580,±1.56,0.836364,9.769477e-02,"[[173, 1413], [225, 1537]]"
0,0,WCunsafe,"children under five, Abidjan",31.578947,±14.78,23.469388,±8.39,1.505017,3.836411e-01,"[[12, 26], [23, 75]]"
1,0,WCdirtyS,"children under five, Abidjan",46.511628,±14.91,17.204301,±7.67,4.184783,6.562460e-04,"[[20, 23], [16, 77]]"


### 3.1.2. Discarded covariate: shared sanitation (by more than 1 household)

In [11]:
# Show ORs for discarded variable
df_oddsr_gen[df_oddsr_gen['exposure variable']=='WCshared'].append(df_oddsr_cU5[df_oddsr_cU5['exposure variable']=='WCshared'])

,index,exposure variable,stratum,"% outcomes, exposed","95% CI, exposed","% outcomes, non-exposed","95% CI, non-exposed",OR,p-value for OR=1,p_OR_hi_1,p_OR_lo_1,table
2,0,WCshared,"general population, Abidjan",15.501520,±2.26,13.545266,±1.76,1.170918,0.176193,0.098245,0.920534,"[[153, 834], [196, 1251]]"
7,0,WCshared,"general population, Nairobi",12.257520,±1.08,12.500000,±6.62,0.977892,0.875378,0.604360,0.519241,"[[436, 3121], [12, 84]]"
2,0,WCshared,"children under five, Abidjan",26.086957,±8.03,23.493976,±6.45,1.149321,0.673069,0.359726,0.738850,"[[30, 85], [39, 127]]"
7,0,WCshared,"children under five, Nairobi",26.282051,±3.99,33.333333,±30.8,0.713043,0.704270,0.808747,0.439267,"[[123, 345], [3, 6]]"


### 3.1.3. Confirm selection of independent variables: test for multicollinearity with Variance Inflation Factor
Note: as a thumbrule, variables with a VIF > 5 shall be discarded 

In [12]:
# Set list of selected exposure + control variables
exposure_lst = ['WCunsafe', # lack of safety to go to most used toilet
                'WCdirtyS' # most used toilet is considered dirty by at least 1/2 of respondents
               ]
control_list = ['ImprvSan', # access to improved sanitation facility
                'BasicHyg', # presence of hand-washing materials AND fixed structure to wash hands (observed)
                #'WCshared', # toilet shared by more than one household
                'StFood', # frequent consumption of street food
                'SecEduHH' # whether head of household has secondary education
                ]
exposure_lst = sorted(set(control_list+exposure_lst))

# Dummy dataframes: keep only observations with valid answers for all selected variables
data_abidjan = df_abi[['Case']+exposure_lst].copy().dropna()
data_nairobi = df_nai[['Case']+exposure_lst].copy().dropna()
data_strings = ['Abidjan','Nairobi']

# Create table for Variance Inflation Factor (VIF)
vif_scores = pd.DataFrame() 
vif_scores["Attribute"] = exposure_lst

for idx,dataset in enumerate([data_abidjan,data_nairobi]):
    # Calculate VIF for each feature
    vif_var = "VIF Scores "+data_strings[idx]
    vif_scores[vif_var] = [variance_inflation_factor(dataset[exposure_lst].values, i) for i in range(len(dataset[exposure_lst].columns))] 
    # Set VIF threshold
    vif_t = 5
    if vif_scores[vif_var].max() < vif_t:
        print("All good: no significant multicollinearity",
              " in ",data_strings[idx])
    else:
        print(">> WARNING: multicollinearity detected for:",
              vif_scores["Attribute"][vif_scores[vif_var]>vif_t].to_list(),
              " in ",data_strings[idx])

# View results
display(vif_scores)

All good: no significant multicollinearity  in  Abidjan
All good: no significant multicollinearity  in  Nairobi


,Attribute,VIF Scores Abidjan,VIF Scores Nairobi
0,BasicHyg,1.585433,1.052516
1,ImprvSan,1.446830,2.117263
2,SecEduHH,1.450769,1.648899
3,StFood,2.124170,2.378513
4,WCdirtyS,1.329249,1.210528
5,WCunsafe,1.372290,1.327013


## 3.2. Adjusted odds ratios (i) : multiple logistic regressions stratified by age group

In [13]:
# Set outcome of interest
outcome = 'Case'

# Stratify by age group
data_reg_genpp_abi = df_abi[[outcome]+exposure_lst].copy().dropna()
data_reg_cu5_abi = df_U5_abi[[outcome]+exposure_lst].copy().dropna()
data_reg_genpp_nai = df_nai[[outcome]+exposure_lst].copy().dropna()
data_reg_cu5_nai = df_U5_nai[[outcome]+exposure_lst].copy().dropna()
data_strata = ['general population, Abidjan ( n = '+str(data_reg_genpp_abi.shape[0])+' )',
               'under-fives subset, Abidjan ( n = '+str(data_reg_cu5_abi.shape[0])+' )',
               'general population, Nairobi ( n = '+str(data_reg_genpp_nai.shape[0])+' )',
               'under-fives subset, Nairobi ( n = '+str(data_reg_cu5_nai.shape[0])+' )']

# Create table for ORs (initially empty)
or_strage = pd.DataFrame()

# Discard exposure variable if all observations are exposed, or non-exposed (OR not plausible)
drop_list = []
print('==============================================================================')
print('                         CHECKING EXPOSURE VARIABLES:')
for data_reg in [data_reg_genpp_abi,data_reg_cu5_abi,
                 data_reg_genpp_nai,data_reg_cu5_nai]:
    print('')
    for var in exposure_lst:
        if len(data_reg[var].value_counts().values)==1:
            drop_list = drop_list+[var]
            print('>> WARNING : dropped',var,'( only 0s or 1s in ',data_strata[idx],' )')
if len(drop_list)==0:
    print('>> All good!')
elif len(drop_list)>0:
    exposure_lst = sorted(list(set(exposure_lst)-set(drop_list))) # update exposure variables list if needed
print('==============================================================================')

# Calculate stratified, adjusted ORs
for idx,data_reg in enumerate([data_reg_genpp_abi,data_reg_cu5_abi,data_reg_genpp_nai,data_reg_cu5_nai]):
    # Fit a logistic regression model with all the variables
    logit = sm.Logit(data_reg[outcome], sm.add_constant(data_reg[exposure_lst]))
    result = logit.fit()

    # Print the summary of the model
    print('==============================================================================')
    print('                       ',data_strata[idx])
    print(result.summary())

    # Calculate the odds ratios and confidence intervals
    params = result.params
    conf = result.conf_int()
    signif = result.pvalues
    modelpval = result.llr_pvalue
    odds_ratios = np.exp(params)
    conf_lower = np.exp(conf[0])
    conf_upper = np.exp(conf[1])

    # Print the odds ratios and confidence intervals
    or_strat = pd.DataFrame({'OR': odds_ratios,
                             'signif': signif,
                             'Lower CI': conf_lower,
                             'Upper CI': conf_upper}).reset_index()
    or_strat['Significance'] = ''
    if modelpval < 0.05:
        or_strat['Significance'][or_strat['signif']>=0.1] = 'Not significant'
        or_strat['Significance'][or_strat['signif']<0.1] = '*'
        or_strat['Significance'][or_strat['signif']<0.05] = '**'
        or_strat['Significance'][or_strat['signif']<0.01] = '***'
        or_strat['Significance'][or_strat['signif']<0.001] = '****'
    else:
        or_strat['Significance'] = 'Not significant (LLR p-value > 5%)'
    or_strat = or_strat.drop(['signif'],axis=1)
    multi_index = pd.MultiIndex.from_tuples([(data_strata[idx],'Exposure'),
                                             (data_strata[idx],'Adjusted OR'),
                                             (data_strata[idx],'Lower CI (95%)'),
                                             (data_strata[idx],'Upper CI (95%)'),
                                             (data_strata[idx],'Significance')])
    or_strat.columns = multi_index
    or_strat = or_strat.iloc[1:]
    or_strage = pd.concat([or_strage,or_strat],axis=1)

# See final OR table
print('==============================================================================')
print('                     Stratified, adjusted odds ratios :')
or_strage = or_strage.replace({'ImprvSan':'Access to improved sanitation facility',
                               'BasicHyg':'Access to basic hygiene amenities',
                               'SecEduHH':'Head of household with secondary education',
                               'StFood':'Frequent consumption of street food',
                               'WCdirtyS':'Toilet considered dirty by most users',
                               'WCunsafe':'Lack of safety to use toilet'
                              })

# Export adjusted OR table
or_strage.to_csv(path.join(root_dir,
                           "data/outputs/odds_ratios/df_OR_diarr_adj4vars.csv"))


# View result
or_strage

                         CHECKING EXPOSURE VARIABLES:




>> All good!
Optimization terminated successfully.
         Current function value: 0.433943
         Iterations 6
                        general population, Abidjan ( n = 942 )
                           Logit Regression Results                           
Dep. Variable:                   Case   No. Observations:                  942
Model:                          Logit   Df Residuals:                      935
Method:                           MLE   Df Model:                            6
Date:                Mon, 20 Feb 2023   Pseudo R-squ.:                 0.05114
Time:                        11:51:46   Log-Likelihood:                -408.77
converged:                       True   LL-Null:                       -430.81
Covariance Type:            nonrobust   LLR p-value:                 7.184e-08
                 coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------

general population, Abidjan ( n = 942 )                             \
                                     Exposure Adjusted OR Lower CI (95%)   
1           Access to basic hygiene amenities    0.575297       0.392350   
2      Access to improved sanitation facility    1.272568       0.875771   
3  Head of household with secondary education    1.363533       0.930375   
4         Frequent consumption of street food    1.451889       0.959321   
5       Toilet considered dirty by most users    1.840825       1.282325   
6                Lack of safety to use toilet    1.896091       1.289228   

                                      under-fives subset, Abidjan ( n = 106 )  \
  Upper CI (95%)     Significance                                    Exposure   
1       0.843551              ***           Access to basic hygiene amenities   
2       1.849147  Not significant      Access to improved sanitation facility   
3       1.998358  Not significant  Head of household with secondary education   
4       2.197370                *         Frequent consumption of street food   
5       2.642573             ****       Toilet considered dirty by most users   
6       2.788615              ***                Lack of safety to use toilet   

                                                              \
  Adjusted OR Lower CI (95%) Upper CI (95%)     Significance   
1    0.306784       0.107247       0.877567               **   
2    2.477117       0.937563       6.544741                *   
3    0.977023       0.343812       2.776447  Not significant   
4    1.491349       0.514285       4.324693  Not significant   
5    2.491900       0.931172       6.668551                *   
6    0.774506       0.243605       2.462423  Not significant   

     general population, Nairobi ( n = 1899 )                             \
                                     Exposure Adjusted OR Lower CI (95%)   
1           Access to basic hygiene amenities    0.760907       0.408224   
2      Access to improved sanitation facility    0.917570       0.673100   
3  Head of household with secondary education    1.078889       0.803415   
4         Frequent consumption of street food    1.238713       0.899923   
5       Toilet considered dirty by most users    1.569651       1.108015   
6                Lack of safety to use toilet    1.690093       1.221398   

                                      under-fives subset, Nairobi ( n = 250 )  \
  Upper CI (95%)     Significance                                    Exposure   
1       1.418290  Not significant           Access to basic hygiene amenities   
2       1.250830  Not significant      Access to improved sanitation facility   
3       1.448818  Not significant  Head of household with secondary education   
4       1.705046  Not significant         Frequent consumption of street food   
5       2.223619               **       Toilet considered dirty by most users   
6       2.338644              ***                Lack of safety to use toilet   

                                             \
  Adjusted OR Lower CI (95%) Upper CI (95%)   
1    2.900314       0.675409      12.454407   
2    1.025482       0.556988       1.888037   
3    0.865017       0.491273       1.523094   
4    1.722574       0.934235       3.176140   
5    1.719274       0.874764       3.379086   
6    0.879379       0.442376       1.748076   

                                       
                         Significance  
1  Not significant (LLR p-value > 5%)  
2  Not significant (LLR p-value > 5%)  
3  Not significant (LLR p-value > 5%)  
4  Not significant (LLR p-value > 5%)  
5  Not significant (LLR p-value > 5%)  
6  Not significant (LLR p-value > 5%)

# 4. Notes

###  On top of safety, hygiene status of toilets seems crucial
The perceived safety to access toilets was not significant - or even relevant - for children under five years, certainly because:
- young children are not necessarily actual users of the toilets analysed, and certainly recur more often to home-based solutions (diapers/bucket), which constitute potential sources of contamination, regardless of the safety and quality of toilets commonly used by the household
- In Nairobi, households with children were more likely to use a WC inside premises (see Notebook 1a)

WC hygiene, however, was consistently associated high higher risks of diarrhoea across age groups, certainly because pathogens may be brought from the toilet to home and, in this way, infect other household members

### Hygiene amenities are effective in Abidjan, but not in Nairobi
In Nairobi, the logistic model even lost significance (lower LLR p-value, see results in cell below) when including presence of basic hygiene amenities in the household